<a href="https://colab.research.google.com/github/Patrick-ChenKehan/BE-521-Project/blob/main/truetest_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithm for BE 5210 Competition -- MONITR

**Upload the submitted `Algorithm_MONITR.zip` and the hidden test set `truetest_data.mat` to colab `/content/`.**

After uploading the files, directly running the notebook will generate the required `predictions.mat`.

## Unzip the Algorithm Files 



*   `models/` includes all saved model necessary for prediction
*   `utils/utils.py` includes all utility functions and feature constructing functions
*    `utils/NN_Model.py` includes the dataset class and the MLP class



In [1]:
!unzip /content/Algorithm_MONITR.zip

Archive:  /content/Algorithm_MONITR.zip
   creating: models/
  inflating: models/train_std_S1.npy  
  inflating: models/train_std_S2.npy  
  inflating: models/train_std_S3.npy  
  inflating: models/NN_S3.pth        
  inflating: models/lgbr_f1_S3.txt   
  inflating: models/idx_S1.npy       
  inflating: models/XGB_S1.json      
  inflating: models/lgbr_f1_S2.txt   
  inflating: models/NN_S2.pth        
  inflating: models/idx_S2.npy       
  inflating: models/idx_S3.npy       
  inflating: models/lgbr_f1_S1.txt   
  inflating: models/NN_S1.pth        
  inflating: models/lgbr_f3_S1.txt   
  inflating: models/lgbr_f3_S3.txt   
  inflating: models/lgbr_f3_S2.txt   
  inflating: models/XGB_S3.json      
  inflating: models/XGB_S2.json      
  inflating: models/lgbr_f0_S1.txt   
  inflating: models/lgbr_f0_S2.txt   
  inflating: models/lgbr_f0_S3.txt   
  inflating: models/lgbr_f2_S2.txt   
  inflating: models/train_mean_S2.npy  
  inflating: models/train_mean_S3.npy  
  inflating: models/

## Import and install related libraries

In [2]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
import xgboost as xgb
from utils import *
from xgboost import XGBRegressor
import lightgbm
import torch

#Load data

Upload the hidden test set `truetest_data.mat` to `/content/`.

In [3]:
raw = scipy.io.loadmat('./truetest_data.mat')
ecog_1 = raw['truetest_data'][0][0]
ecog_2 = raw['truetest_data'][1][0]
ecog_3 = raw['truetest_data'][2][0]

## Compute features and R_Matrix

In [4]:
# Set up params for windows
winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap

# Compute features
feature_1 = get_windowed_feats(ecog_1, 1000, winLen, winOverlap)
feature_2 = get_windowed_feats(ecog_2, 1000, winLen, winOverlap)
feature_3 = get_windowed_feats(ecog_3, 1000, winLen, winOverlap)

# Load feature indices
idx_1 = np.load('./models/idx_S1.npy')
idx_2 = np.load('./models/idx_S2.npy')
idx_3 = np.load('./models/idx_S3.npy')

# Compute R_matrix of 20 windows and select features with indices
R_1 = create_R_matrix(feature_1, 20)[:, idx_1]
R_2 = create_R_matrix(feature_2, 20)[:, idx_2]
R_3 = create_R_matrix(feature_3, 20)[:, idx_3]
R_list = [R_1, R_2, R_3]

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  warnings.warn('nperseg = {0:d} is greater than input length '


## Load model and Prediction

In [5]:
# Load mean and std of the train data for normalization

train_mean_S1 = np.load('./models/train_mean_S1.npy')
train_std_S1 = np.load('./models/train_std_S1.npy')

train_mean_S2 = np.load('./models/train_mean_S2.npy')
train_std_S2 = np.load('./models/train_std_S2.npy')

train_mean_S3 = np.load('./models/train_mean_S3.npy')
train_std_S3 = np.load('./models/train_std_S3.npy')

train_mean_ls = [train_mean_S1, train_mean_S2, train_mean_S3]
train_std_ls = [train_std_S1, train_std_S2, train_std_S3]

device = torch.device("cpu") # Using CPU for convenience

In [6]:
predictions = []

for i in range(3): # for each subject
    
    # Load LGBM for each finger and predict
    lgbm_reg_list = [lightgbm.Booster(model_file=f'./models/lgbr_f{j}_S{i + 1}.txt') for j in range(4)]
    
    prediction_lgbm_list = [lgbm_reg.predict(R_list[i]) for lgbm_reg in lgbm_reg_list]
    prediction_lgbm = np.vstack(prediction_lgbm_list).T
    
    # Load normalized data into the dataloader
    leaderboard_dataset = FingerFeatureDataset((R_list[i] - train_mean_ls[i]) / train_std_ls[i], np.zeros(R_list[i].shape[0]).copy())
    dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

    # Load MLP and predict
    net = FingerRegressor(R_list[i].shape[1], 4).to(device)
    net.load_state_dict(torch.load(f'./models/NN_S{i + 1}.pth', map_location=device))
    
    with torch.no_grad():
        pred = []
        net.eval()
        for i, (ecog, dg) in enumerate(dataloader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]

    prediction_NN = np.concatenate(pred)
    
    # Ensemble the NN prediction and LightGBM prediction by averaging
    prediction = (prediction_lgbm + prediction_NN) / 2
    predictions.append(prediction)

## Pack prediction to file for submission

In [7]:
pack_submission(predictions, "predictions.mat", "predicted_dg")

In [8]:
# Check submission file
raw = scipy.io.loadmat('./predictions.mat')
raw['predicted_dg'].shape

(3, 1, 147500, 5)